In [ ]:
%matplotlib widget
## Distributed python packages
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import datetime
import pandas as pd
import time as ttime

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from math import radians, log

## Own python package
from synthetics import *

%matplotlib inline
__author__ = "Lisa Tomasetto"
__copyright__ = "Copyright 2024, UGA"
__credits__ = ["Lisa Tomasetto"]
__version__ = "1.0"
__maintainer__ = "Lisa Tomasetto"
__email__ = "lisa.tomasetto@univ-grenoble-alpes.fr"
__status__ = "Developpement"

## Matplotlib Parameters

In [ ]:
plt.style.use("ggplot")
SMALL_SIZE = 18
MEDIUM_SIZE = 22
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Theory 

We will compute synthetic cross-correlations in the following Notebook. To that aim we express the cross-correlation function as derived from the representation theorem, assuming spatially uncorrelated sources. For two stations A and B at position $ x_A $ and $ x_B $ respectively, the cross-correlation between them can be written:
$$ C(x_A, x_B, t) = \mathcal{FT}^{-1} \left[ \int_{\delta D} G(x_A, x, f) G^*(x_B, x, f) S(x, f) dx\right] $$

with:
- $ G(x_A, x, f) $ the Green's Function between a source at $ x $ and station A at $ x_A $. We use Syngine tool to compute a data archive in the first paragraph of this Notebook.
- $ ^* $ denotes the complex conjugate.
- $ S(x, f) $ the source PSD, derived from the equivalent vertical force amplitude here.
- $ t $ the time in s, $ f $ the seismic frequency in Hz.

## Create Synthetic Waveforms Database Using Syngine

# RUN ONCE !!

In [ ]:
## Create station file
if os.path.exists('station_file.txt'):
    print('Station file already exists')
else:
    create_station_file()

In [ ]:
## Create GF Database with Syngine
if os.path.exists('synthetics_iasp91_2s_Z.nc'):
    print('GF archive already exists')
else:
    create_syngine_archive(station_file_path ='station_file.txt')

In [ ]:
## Create Spectrum Database
if os.path.exists('synthetics_iasp91_2s_Z_spectrum.h5'):
    print('Spectrum archive already exists')
else:
    create_spectrum_archive() 

# Synthetics Computation

## Spatial Extent

In [ ]:
## Spatial Exent
lon_min = -150
lon_max = -20
lat_min = 0
lat_max = 70

extent = [lon_min, lon_max, lat_min, lat_max]

## Station Pairs and Metadata

In [ ]:
## Station Pairs Metadata
df = pd.read_csv('stations_pair.txt', sep='   ', header=None, index_col=False, names=['idA', 'idB', 'latA', 'latB', 'lonA', 'lonB'], dtype={'idA':str, 'idB':str, 'latA':float, 'latB':float, 'lonA':float, 'lonB':float}, engine='python')
# ## Plot
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=ccrs.Mercator())
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', alpha=1)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.LAND, color='linen', zorder=0)
ax.add_feature(cfeature.OCEAN, color='lightsteelblue', zorder=0)
for i in range(len(df)):
    ax.annotate(df.idA[i], (df.lonA[i]+0.1, df.latA[i]+0.1), transform = ccrs.PlateCarree(), family = 'sans-serif')
    ax.annotate(df.idB[i], (df.lonB[i]+0.1, df.latB[i]+0.1), transform = ccrs.PlateCarree(), family = 'sans-serif')
    ax.plot((df.lonA[i], df.lonB[i]),(df.latA[i], df.latB[i]), transform = ccrs.PlateCarree(), color='k', alpha=0.5, linewidth=3)
df.plot.scatter(x='lonA', y='latA', s=30, ax=ax, c='r', transform = ccrs.PlateCarree())
df.plot.scatter(x='lonB', y='latB', s=30, ax=ax, c='r', transform = ccrs.PlateCarree())
plt.show()

## Dates 

In [ ]:
## Dates
start = datetime(1996, 3, 11, 0, 0, 0)
end = datetime(1996, 4, 9, 0, 0, 0)

dates = pd.date_range(start, end, freq='3H')

dates_vector = create_date_vect(dates)

## Open WAVEWATCHIII Equivalent Force Amplitude

In [ ]:
## Open Model
path_model = '/Users/tomasetl/Documents/code/ocean_source/pr_oceanswell/rayleigh/F/'

## Open Syngine Archive 

In [ ]:
## Open GF archive info
fe , freq, time, distance, spectrum_synth = open_archive(h5_name_spectrum='./synthetics_iasp91_2s_Z_spectrum.h5', h5_name_waveforms='./synthetics_iasp91_2s_Z_waveforms.h5')
N_time = len(time)

## Synthetic Cross-correlation Computation

In [ ]:
## Compute CCF and save it
### Station Pair Info
id_A = df.idA[0]
id_B = df.idB[0]
coords_staA = (df.lonA[i], df.latA[i])
coords_staB = (df.lonB[i], df.latB[i])

### Date Info
date_vect = dates_vector[0]

### CCF
filename_ccf = './CCF/ccf_%s_%s_%d%02d%02d%02d.nc'%(id_A, id_B, date_vect[0],date_vect[1],date_vect[2],date_vect[3]) # File Name Understandable

if os.path.exists(filename_ccf):  # Check if file already exists
    print('%s already computed'%filename_ccf)
    corr = xr.open_dataarray(filename_ccf)
else:  # if not, compute CCF
    t1 = ttime.perf_counter()
    corr, time_corr = ccf_computation(coords_staA, coords_staB, path_model, date_vect, spectrum_synth,extent=extent, fe=fe, N=N_time, comp='Z')
    t2 = ttime.perf_counter()
    print(" Time for station pair: ", id_A, id_B, " elapsed: ", t2-t1)
    

## Plot

In [ ]:
corr.to_netcdf('./CCF/ccf_%s_%s_%d%02d%02d%02d.nc'%(id_A, id_B, date_vect[0],date_vect[1],date_vect[2],date_vect[3]))
## Plot CCF
corr.plot()
plt.show()